## Matrix multiplication from foundations

The *foundations* we'll assume throughout this course are:

- Python
- Python modules (non-DL)
- pytorch indexable tensor, and tensor creation (including RNGs - random number generators)
- fastai.datasets

##### ALGComment: why these primitives?

It's not obvious to me why "indexable tensors" are treated as a foundational block, rather than a structured we build from something more primitive like an array.

Is that because they're hard to build?

## Check imports

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export
from exp.nb_00 import *
import operator

def test(a,b,cmp,cname=None):
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

def test_eq(a,b): test(a,b,operator.eq,'==')

In [ ]:
test_eq(TEST,'test')

In [ ]:
# To run tests in console:
# ! python run_notebook.py 01_matmul.ipynb

## Get data

In [ ]:
#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'

In [ ]:
path = datasets.download_data(MNIST_URL, ext='.gz'); path

In [ ]:
with gzip.open(path, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [ ]:
x_train,y_train,x_valid,y_valid = map(tensor, (x_train,y_train,x_valid,y_valid))
n,c = x_train.shape
x_train, x_train.shape, y_train, y_train.shape, y_train.min(), y_train.max()

##### AlGComment: These dimensions ...

... are what we expect from 50,000 training sample images, each 28x28 greyscale, being classified into categories labeled 0...9. To spell it out, the trainin data tensor is 50k array of items each 784 long.

In [ ]:
assert n==y_train.shape[0]==50000
test_eq(c,28*28)
test_eq(y_train.min(),0)
test_eq(y_train.max(),9)

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
img = x_train[0]

In [ ]:
img.view(28,28).type()

In [ ]:
plt.imshow(img.view((28,28)));

## Initial python model

##### ALGComment: why these values?

This is the "initial Python model" because we're going to refine either the implementation or the complexit of the model as we go.

784 is the number or floats in one image. So 784x10 gives us sufficient weights for a matriz that takes one image as input (784 rows) and outputs a vector with 10 elements (10 columns). These 10 elements will be values indicating the relative likelihood of the 10 labels, presumably.

We start with random values either because we're testing our compute code wih realistic values or because we want a non-symmetric, non-degenerate initial value to start training drom.

In [ ]:
weights = torch.randn(784,10)

In [ ]:
bias = torch.zeros(10)

ALGComment: the bias is the offset value in our initial model which is just offset + M * x

#### Matrix multiplication

In [ ]:
def matmul(a,b):
    ar,ac = a.shape # n_rows * n_cols
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): # or br
                c[i,j] += a[i,k] * b[k,j]
    return c

In [ ]:
m1 = x_valid[:5]
m2 = weights

##### ALGComment: x_valid ...

... is the validation data. It's like the training data (50000,784) but with fewer samples. So the splice above, just takes the first 5 items from that already smaller set.

In [ ]:
x_valid.shape

In [ ]:
m1.shape,m2.shape

##### ALGComment: The following product ...

... is done to benchmark perf. But is still meaningful.

This just represents doing the multiplication part of the model to 5 samples, so the resulting value is, for each sample, a length-10 vector indicating the likilihood of the 10 labels.

In [ ]:
%time t1=matmul(m1, m2)

In [ ]:
t1.shape

This is kinda slow - what if we could speed it up by 50,000 times? Let's try!

In [ ]:
len(x_train)

#### Elementwise ops

Operators (+,-,\*,/,>,<,==) are usually element-wise.

Examples of element-wise operations:

In [ ]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a,b

In [ ]:
a + b

In [ ]:
(a < b).float().mean()

##### ALGComment: lets verify elementwise < gives 0 or 1

In [ ]:
(a < b).min(), (a < b).max()

In [ ]:
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]]); m

Frobenius norm:

$$\| A \|_F = \left( \sum_{i,j=1}^n | a_{ij} |^2 \right)^{1/2}$$

*Hint*: you don't normally need to write equations in LaTeX yourself, instead, you can click 'edit' in Wikipedia and copy the LaTeX from there (which is what I did for the above equation). Or on arxiv.org, click "Download: Other formats" in the top right, then "Download source"; rename the downloaded file to end in `.tgz` if it doesn't already, and you should find the source there, including the equations to copy and paste.

In [ ]:
(m*m).sum().sqrt()

#### Elementwise matmul

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            # Any trailing ",:" can be removed
            c[i,j] = (a[i,:] * b[:,j]).sum()
    return c

In [ ]:
%timeit -n 10 _=matmul(m1, m2)

In [ ]:
890.1/5

##### ALGComment: my perf difference was greater

In [ ]:
682.0/0.972

In [ ]:
#export
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
def test_near(a,b): test(a,b,near)

In [ ]:
test_near(t1,matmul(m1, m2))

The above ensures that the Python matmul is near the value of the matmul using elementwise multiplication in the innermost loop

### Broadcasting

The term **broadcasting** describes how arrays with different shapes are treated during arithmetic operations.  The term broadcasting was first used by Numpy.

From the [Numpy Documentation](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.broadcasting.html):

    The term broadcasting describes how numpy treats arrays with 
    different shapes during arithmetic operations. Subject to certain 
    constraints, the smaller array is “broadcast” across the larger 
    array so that they have compatible shapes. Broadcasting provides a 
    means of vectorizing array operations so that looping occurs in C
    instead of Python. It does this without making needless copies of 
    data and usually leads to efficient algorithm implementations.
    
In addition to the efficiency of broadcasting, it allows developers to write less code, which typically leads to fewer errors.

*This section was adapted from [Chapter 4](http://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/4.%20Compressed%20Sensing%20of%20CT%20Scans%20with%20Robust%20Regression.ipynb#4.-Compressed-Sensing-of-CT-Scans-with-Robust-Regression) of the fast.ai [Computational Linear Algebra](https://github.com/fastai/numerical-linear-algebra) course.*

#### Broadcasting with a scalar

In [ ]:
a

In [ ]:
a > 0

How are we able to do a > 0?  0 is being **broadcast** to have the same dimensions as a.

For instance you can normalize our dataset by subtracting the mean (a scalar) from the entire data set (a matrix) and dividing by the standard deviation (another scalar), using broadcasting.

Other examples of broadcasting with a scalar:

In [ ]:
a + 1

In [ ]:
m

In [ ]:
2*m

#### Broadcasting a vector to a matrix

We can also broadcast a vector to a matrix:

In [ ]:
c = tensor([10.,20,30]); c

In [ ]:
m

In [ ]:
m.shape,c.shape

In [ ]:
m + c

In [ ]:
c + m

We don't really copy the rows, but it looks as if we did. In fact, the rows are given a *stride* of 0.

In [ ]:
t = c.expand_as(m)

In [ ]:
t

In [ ]:
?t.expand_as

In [ ]:
m + t

In [ ]:
t.storage()

In [ ]:
c.storage()

##### ALGComment: the comparison of c and t is the key point w/r/t/ storage.

The method `expand_as` changes the _shape_ of the structure, from rank 1 to rank 2, but not the storage.

I infer that `expand_as` is an explicit form of what broadcasting does implicitly when an operation requires it.

Broadcasting provides a storage and expressivity benefit.

In [ ]:
?t.storage

In [ ]:
t.stride(), t.shape

In [ ]:
c.stride(), c.shape

You can index with the special value [None] or use `unsqueeze()` to convert a 1-dimensional array into a 2-dimensional array (although one of those dimensions has value 1).

In [ ]:
?torch.unsqueeze

In [ ]:
c.unsqueeze(0)

In [ ]:
c.unsqueeze(1)

In [ ]:
m

In [ ]:
c.shape, c.unsqueeze(0).shape,c.unsqueeze(1).shape

In [ ]:
c.shape, c[None].shape,c[:,None].shape

ALGC: Question: whats the relationship between the expansion automatically done for broadcasting, the one explicitly done by `expand_as`, and the one done by `unsqueeze`?

Seems like: unsqueeze is exactly synonymous with None-indexing.

In [ ]:
c.expand_as(m).shape

##### ALGC: unsqueezing versus expanding

`expand_as` not only increases the _rank_ of the tensor, but duplicates values along the new dimension as needed in order to create a _larger_ shape.

`unsqueeze` (and None-indexing) increases the _rank_ by adding a new dimension, reshaping the tensor, but does not add new values, since it adds a new dinension of size 1.

You can always skip trailling ':'s. And '...' means '*all preceding dimensions*'

In [ ]:
c[None].shape,c[...,None].shape

In [ ]:
c[:,None].expand_as(m)

In [ ]:
m + c[:,None]

In [ ]:
c[:,None]

#### Matmul with broadcasting

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:]          * b[:,j]).sum() # previous
        c[i]   = (a[i  ].unsqueeze(-1) * b).sum(dim=0)
    return c

##### <ALGC: Why does this work?

Let's look at its pieces.

When we used elementwise multiplication, we used it for the inner product performed for every element in the product matrix. but we still looped over every row of A and every column of B.

This new formulation only loops over every row of A.

Then it seems to build the product matrix one row at a time.

So the broadcasting operation loops implicitly operate over every column of B. But operating over every column of B sounds like matrix multiplication, which is what we're trying to implement. Are we cheating? no.

The expression `a[i  ].unsqueeze(-1)` reshapes a single row of A (`[784]`) into a matrix (`[784,1]`), so it's a tensor with the same rank as B (`[784,10]`). The multiplication `(a[i  ].unsqueeze(-1) * b)` then does two things: it implicitly broadcasts that matrix expanding its number of columns so it has the same shape as B (`[784,10]`), and then performs an _elementwise_ multiplication over those two tensors. Finally, summing over the 0th dimension does the inner product for all of B's columns in one operation, and every resulting element is the sum of a column.



In [ ]:
m1.shape,m2.shape

In [ ]:
m1[0].shape

In [ ]:
m1[0].unsqueeze(-1).shape

In [ ]:
(m1[0].unsqueeze(-1) * m2).shape

In [ ]:
(m1[0].unsqueeze(-1) * m2).sum(dim=0)

##### ALGC>

In [ ]:
%timeit -n 10 _=matmul(m1, m2)

In [ ]:
885000/277

In [ ]:
test_near(t1, matmul(m1, m2))

#### Broadcasting Rules

In [ ]:
c[None,:] # add a size=1 dimension at rank index 0

In [ ]:
c[None,:].shape

In [ ]:
c[:,None] # add a size=1 dimension at rank index 1

In [ ]:
c[:,None].shape

##### ALGC: The following is _not_ a matrix multiplication

Rather, `([1,3] * [1,3])` broadcasts both sides to make two `[3,3]` matrixes and then multiplies them elementwise.

In [ ]:
c[None,:] * c[:,None]

In [ ]:
c[None] > c[:,None]

When operating on two arrays/tensors, Numpy/PyTorch compares their shapes element-wise. It starts with the **trailing dimensions**, and works its way forward. Two dimensions are **compatible** when

- they are equal, or
- one of them is 1, in which case that dimension is broadcasted to make it the same size

Arrays do not need to have the same number of dimensions. For example, if you have a `256*256*3` array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

    Image  (3d array): 256 x 256 x 3
    Scale  (1d array):             3
    Result (3d array): 256 x 256 x 3

The [numpy documentation](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html#general-broadcasting-rules) includes several examples of what dimensions can and can not be broadcast together.

##### ALGC: `unsqueeze`, `expand_as`, broadcasting

It is not clear to me _exactly_ what is the relationship between `unsqueeze`, `expand_as`, None-indexing, and the rules that govern broadcasting. Let me try to state them here completely and unambiguously and with examples.

**expand_as** simply expands one tensor to the same size as another. It seems to rely on `expand`, which expands a tensor to a new, explicitly specified shape.

In [ ]:
myR0 = tensor(1)
myR0, myR0.shape, myR0.size()

In [ ]:
myR1 = tensor([7,8])
myR1, myR1.shape, myR1.size()

In [ ]:
myR2 = tensor([[1,2,3],[4,5,6]])
myR2, myR2.shape

`torch.expand` takes a shape vector, and expands the argument so that it has that shape. It duplicates elements as needed to do this.

A scalar has shape `[]`, aka (rank 0).

If we expand the `[]` shape to `[1]`, we're expanding it to be a length-1 vector (rank 1). We increasing the _rank_ from 0 to 1 but not changing what you might call the total _volume_ of the shape, the number of float values it specifies.


In [ ]:
myR0, myR0.shape

In [ ]:
myR0.expand([1]), myR0.expand([1]).shape

If we expand the `[]` shape to `[10]`, we're expanding it to be a length-10 vector (rank 1).

We increasing the _rank_ from 0 to 1 and we are also changing the total _volume_ of the shape, by going from 1 to 10 values.

In [ ]:
myR0.expand([10]), myR0.expand([10]).shape

If we expand the `[]` shape to `[4,4]`, we're expanding it to be a 4x4 matrix (rank 2), by duplicating the value 16 times.

We are increasing the rank from 0 to 2 and increasing the volume as well

In [ ]:
myR0.expand([4,4]), myR0.expand([4,4]).shape

Only certain expansions are permitted.

For instance, if we started with a length-2 vector (shape `[2]`, rank 1), then:

- we cannot simply expand to `[3]`


In [ ]:
tensor([7,8]).expand([3])

Why not? This would be merely extending the length-2 vector by adding a new element along the existing dimension. But what should this new element be? Should the new element be the first or the second of our existing two elements?

The operation _expand_ does not encompass any rule to answer that sort of question. In other words, _expand does not specify how to expand along an existing dimension when there's more than one way to do it._

There is only one way to do the expansion along a dimension, when there is only one possible value to use for expansion -- in other words, when that dimension currently has a size of 1. This is what the error message means when it refers to a "singleton dimension".

For instance, if we started with a length-2 vector (shape `[2]`, rank 1), then:

- we cannot simply expand to `[2,3]`



In [ ]:
myR1, myR1.shape

In [ ]:
myR1.expand([2,3])

Why does this fail? This would take a length-2 vector (shape `[2]`) and expand it to a 2 row, 3 column matrix (shape `[2,3]`). 

$ \begin{bmatrix}
a & a & a \\
b & b & b 
\end{bmatrix}  $

You cannot do this.



In [ ]:
myR1.expand([1,3]), myR1.expand([1,3]).shape

In [ ]:
myR1.expand([3,2]), myR1.expand([3,2]).shape

But you _can_ expand to a `[3,2]` shape:

$ \begin{bmatrix}
a & b \\
a & b \\
a & b 
\end{bmatrix}  $

This difference seems arbitrary. Why the discrepancy? Is there an intuition for it that makes it logical or at least memorable?

One way to make sense of this would be to suppose that a vector (rank 1 tensor) is "really" a row:

$ \begin{bmatrix}
a & b 
\end{bmatrix}  $

... and that you are only allowed to expand by duplicating that row to add the new dimension of columns. This perspective is consistent with how a rank-1 and rank-2 tensors are written as nested arrays. However, it seems to me inconsistent with our usual view that when we multiply a matrix on the left by a vector on the right, then that vector was already a "column vector".

This is confusing.

Perhaps the only reliable way to make sense of this is just to learn the formal rule for when expansion is allowed, which seems to be as follows:

If you're expanding an existing shape `[a_(m),a_(m-1),...,a_1]` to a new shape `[b_(n),b_(n-1),...,b_1]`, then:

- `a_i` needs to "match" `b_i` for i in 1...m
- where "match" means that `a_i` == 1 or else that `a_i` == `b_i`


##### So what is the connection with _broadcasting_?

One way to understand broadcasting seems to be this: when you want to perform an elementwise operation on two tensors of different shapes, then one tensor will be expanded to the shape of the other, if possible, in order to complete the operation.

It is not clear to me if this is equivalent to the broadcasting rule defined above.

##### Exercises: Can you broadcast?


- (`[256,256,3]`,  `[3]`) : yes, because 3 is compatible with 3
- (`[256,3]`,  `[3]`): yes, because 3 is compatible with 3
- (`[256,3]`,  `[1]`): yes, because 1 is compatible with 3
- (`[256,256,3]`,  `[1,3]`): yes, becuse 3 icw 3, and 256 icw 1
- (`[256,256,3]`,  `[1,1,3]`): yes, becuse 3 icw 3, and 256 icw 1
- (`[256,256,3]`,  `[1,256,3]`): yes, becuse 256 icw 256, 256 icw 1





In [ ]:
torch.zeros([256,256,3]) * torch.zeros([1,256,3]);

In [ ]:
myR0.shape,myR1.shape,myR2.shape

In [ ]:
myR1.expand([100,2]).shape

### Einstein summation

Einstein summation (`einsum`) is a compact representation for combining products and sums in a general way. From the numpy docs:

"The subscripts string is a comma-separated list of subscript labels, where each label refers to a dimension of the corresponding operand. Whenever a label is repeated it is summed, so `np.einsum('i,i', a, b)` is equivalent to `np.inner(a,b)`. If a label appears only once, it is not summed, so `np.einsum('i', a)` produces a view of a with no changes."

In [ ]:
# c[i,j] += a[i,k] * b[k,j]
# c[i,j] = (a[i,:] * b[:,j]).sum()
def matmul(a,b): return torch.einsum('ik,kj->ij', a, b)

In [ ]:
%timeit -n 10 _=matmul(m1, m2)

In [ ]:
885000/55

In [ ]:
test_near(t1, matmul(m1, m2))

### pytorch op

We can use pytorch's function or operator directly for matrix multiplication.

In [ ]:
%timeit -n 10 t2 = m1.matmul(m2)

In [ ]:
# time comparison vs pure python:
885000/18

In [ ]:
t2 = m1@m2

In [ ]:
test_near(t1, t2)

In [ ]:
m1.shape,m2.shape

## Export

In [ ]:
!python notebook2script.py 01_matmul.ipynb